In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

import gmaps.datasets
gmaps.configure(api_key=g_key)

In [2]:
# Load the CSV created in Vacation_Search.ipynb into a DataFrame.
vacation_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv", index_col = 0)

In [3]:
vacation_data_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Green River,US,86.00,clear sky,41.53,-109.47,Hampton Inn & Suites Green River
6,Georgetown,MY,80.60,few clouds,5.41,100.34,Cititel Penang
9,Ahuimanu,US,84.20,broken clouds,21.44,-157.84,Paradise Bay Resort
10,Mentor,US,75.99,overcast clouds,41.67,-81.34,Holiday Inn Cleveland Northeast - Mentor
17,Qiongshan,CN,80.60,clear sky,20.01,110.35,Sun City Restaurant
18,Svay Rieng,KH,80.11,overcast clouds,11.09,105.80,Apsara guesthouse
21,Hithadhoo,MV,84.83,broken clouds,-0.60,73.08,Scoop Guest House
29,Bethel,US,81.00,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
34,Roma,US,78.80,clear sky,43.21,-75.46,Wingate By Wyndham Rome
49,Cabo San Lucas,MX,95.00,scattered clouds,22.89,-109.91,Hotel Tesoro Los Cabos


In [4]:
# Convert Max Temp type from float64 to string.
vacation_data_df['Max Temp'] = vacation_data_df['Max Temp'].astype(str)

In [5]:
# Creating a new column with Current Weather by merging Current Description and Max Temp into one.
# Max Temp was converted to a string prior to appending.
vacation_data_df["Current Weather"] = vacation_data_df["Current Description"] +" and "+ vacation_data_df["Max Temp"] +" " u"\N{DEGREE SIGN}""F"

# Dropping the columns for Current Description and Max Temp as the information has been replaced with Current Weather
vacation_data_df = vacation_data_df.drop(['Current Description', 'Max Temp'], axis=1)

# Rearranging the columns in DataFrame
vacation_data_df = vacation_data_df[["City", "Country", "Lat", "Lng", "Current Weather", "Hotel Name"]]

In [6]:
vacation_data_df.head()

,City,Country,Lat,Lng,Current Weather,Hotel Name
3,Green River,US,41.53,-109.47,clear sky and 86.0 °F,Hampton Inn & Suites Green River
6,Georgetown,MY,5.41,100.34,few clouds and 80.6 °F,Cititel Penang
9,Ahuimanu,US,21.44,-157.84,broken clouds and 84.2 °F,Paradise Bay Resort
10,Mentor,US,41.67,-81.34,overcast clouds and 75.99 °F,Holiday Inn Cleveland Northeast - Mentor
17,Qiongshan,CN,20.01,110.35,clear sky and 80.6 °F,Sun City Restaurant


In [7]:
# Creating a pop up marker for the DataFrame to allow layering on a map.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
</dl>
"""

In [8]:
# Store the DataFrame Row.
vacation_cities_info = [info_box_template.format(**row) for index, row in vacation_data_df.iterrows()]

In [9]:
# map with selected vacation spots.
locations = vacation_data_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 30.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_cities_info)

#  Adding marker layers.
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Creating DataFrame of the 4 selected cities for travel.
kothi_df = vacation_data_df.loc[(vacation_data_df["City"] == "Kothi")]
bombay_df = vacation_data_df.loc[(vacation_data_df["City"] == "Bombay")]
karwar_df = vacation_data_df.loc[(vacation_data_df["City"] == "Karwar")]
yanam_df = vacation_data_df.loc[(vacation_data_df["City"] == "Yanam")]

In [11]:
yanam_df

,City,Country,Lat,Lng,Current Weather,Hotel Name
63,Yanam,IN,16.73,82.22,overcast clouds and 80.98 °F,The Regency Hotel


In [12]:
# Importing dependable.
import numpy as np

In [13]:
# listing the cities.
kothi = (np.array(kothi_df)[0,2],np.array(kothi_df)[0,3])
bombay = (np.array(bombay_df)[0,2],np.array(bombay_df)[0,3])
karwar = (np.array(karwar_df)[0,2],np.array(karwar_df)[0,3])
yanam = (np.array(yanam_df)[0,2],np.array(yanam_df)[0,3])

In [14]:
# Creating a data frame of the vacation cities.
city_marker_df = pd.concat([kothi_df, bombay_df, karwar_df, yanam_df])

In [15]:
city_marker_df.head()

,City,Country,Lat,Lng,Current Weather,Hotel Name
547,Kothi,IN,24.75,80.78,broken clouds and 82.92 °F,KILA-THE HERITAGE PALACE
447,Bombay,IN,19.01,72.85,light rain and 80.6 °F,Four Seasons Hotel Mumbai
443,Karwar,IN,14.80,74.13,overcast clouds and 79.0 °F,Aditi Comforts
63,Yanam,IN,16.73,82.22,overcast clouds and 80.98 °F,The Regency Hotel


In [16]:
# Selecting travel mode as 'DRIVING' and creating a travel itinerary from Rong Kwang to Sattahip through Loei and Khanu Woralaksaburi.
fig = gmaps.figure()

# Creating a travel marker
india_marker = gmaps.directions_layer(kothi, karwar, waypoints=[bombay, yanam],
        travel_mode='DRIVING')
#  Adding the travel layer.
fig.add_layer(india_marker)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Creating a pop up marker for the DataFrame above to allow layering on a map.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
</dl>
"""

In [18]:
# Store the DataFrame Row.
vacation_info = [info_box_template.format(**row) for index, row in city_marker_df.iterrows()]

In [24]:
# Add marker layer with pop up on map.
locations = city_marker_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(22.0, 80.0), zoom_level=5.0)

#Adding pop-up marker
marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_info)

#  Adding marker.
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))